# Macros
Los mácros son técnicas de metaprogramación que permiten dar un contexto de ejecución al código de un programa. Son faciles de reconocer pues empiezan siempre con un `@`. Por ahora nos concentraremos solamente en utilizarlos.

In [2]:
@time [i^2 for i in 1:2^10 if i%2==0]; # Lista de comprensión condicional

  0.085074 seconds (20.35 k allocations: 1.149 MiB)


In [3]:
println(@time sum([i^2 for i in 1:2^10 if i%2==0]);)
println(@time sum(i^2 for i in 1:2^10 if i%2==0);)

  0.176592 seconds (20.35 k allocations: 1.150 MiB)
179481600
  0.067916 seconds (23.69 k allocations: 1.308 MiB)
179481600


# Compilador e Interprete
Un compilador es el programa que traduce código fuente a lenguaje de máquina. Un interprete hace una interpretación instantanea de un lenguaje de programación y produce una respuesta inmediata. El código de máquina generado por un compilador puede ser guardado y conservado para despues, mientras que un interprete solo puede producir comportamiento instantaneo.

Julia, a pesar de parecer un lenguaje interpretado, por su interactividad, es un lenguaje compilado, su compilador es nombrado JIT (Just in Time) ya que provee código de máquina instantaneamente. Si deseas crear executables puedes revisar el siguiente link: http://juliacomputing.com/blog/2016/02/09/static-julia.html

# Alto desempeño
(Gracias a Ismael Villegas Castelló por éste ejemplo)
El alto desempeño en Julia se debe a su especialización en cada tarea. Para verlo más a fondo utilizaremos la siguiente función:

In [4]:
g() = 10_000_000 
function test()                                             
    i = 0                                                   
        while i < 10_000_000                                    
            i += 1                                              
        end                                                     
    return i                                                
end  

test (generic function with 1 method)

Es posible ver la especialicación en las tareas con la ayuda de los siguientes macros:

## `@code_lowered` 
Código bajado, es decir parseado y aplicado el primer pase del compilador.

In [5]:
@code_lowered test()

CodeInfo(:(begin 
        nothing
        i = 0 # line 4:
        4: 
        unless i < 10000000 goto 10 # line 5:
        i = i + 1
        8: 
        goto 4
        10:  # line 7:
        return i
    end))

## `@code_typed` 
Código bajado y con anotaciones de tipos, por medio de la inferencia automática de tipos, en otro pase del compilador.

In [6]:
@code_typed test()

CodeInfo(:(begin 
        i = 0 # line 4:
        3: 
        unless (Base.slt_int)(i, 10000000)::Bool goto 9 # line 5:
        i = (Base.add_int)(i, 1)::Int64
        7: 
        goto 3
        9:  # line 7:
        return i
    end))=>Int64

## `@code_llvm` 
Código convertido a LLVM IR (Representación Intermedia de la Máquina Virtual de Bajo Nivel. Esto es homologo a un bytecode, sin embargo es de bajo nivel, o dicho de otra manera es una especie de ensamblador de alto nivel. Desde este nivel, podemos ver que el bucle ya ha sido eliminado completamente.

In [7]:
@code_llvm test()


; Function Attrs: uwtable
define i64 @julia_test_62033() #0 !dbg !5 {
top:
  ret i64 10000000
}


# `@code_native`
Finalmente el código generado en ensamblador, el cual posteriormente es compilado a código binario nativo, especifico al procesador donde se ejecutará. Éste es el ultimo paso que podemos ver.

In [8]:
@code_native test()

	.text
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 7
	movl	$10000000, %eax         # imm = 0x989680
	popq	%rbp
	retq
	nopl	(%rax,%rax)


In [9]:
@code_native g()

	.text
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movl	$10000000, %eax         # imm = 0x989680
	popq	%rbp
	retq
	nopl	(%rax,%rax)


# Medición de Rendimiento
Además del funcional y rápido macro `@time`, existe un paquete que nos permite medir con precisión los tiempos de trabajo de Julia: [`BenchmarkTools`](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md):

In [12]:
using BenchmarkTools
@benchmark test()

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.020 ns (0.00% GC)
  median time:      3.398 ns (0.00% GC)
  mean time:        3.753 ns (0.00% GC)
  maximum time:     855.221 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

# Mejorar rendimiento
Julia es sencillo de usar, pero para mejorar rendimiento es útil seguir algunas recomendaciones:

- Al cargar un script, estás cargando el compilador de Julia, por lo que puede tardar en comenzar.

- Si quieres mejorar rendimiento de una función, encapsulala en un módulo y deja que Julia la precompile.

- Jupyter y la terminal son para probar ideas. Si vas a trabajar, trabaja en scripts.

- Usa funciones: No dejes procesos en el scope global.

- Revisa la especialización de tus tipos.

- Deja que Julia "caliente". El compilador tiene que compilar tu código la primera vez que es ejecutado. Despues usará el código nativo.

- Especifica tipos. Evita los arreglos de tipo `Any` y los flotantes cuando solo necesitas enteros.

- Revisa tus tipos. Utiliza el macro `@code_warntype` para evitar usar variables de tipo inespecifico.

In [13]:
pos(x) = x < 0 ? 0 : x

function f(x)
    y = pos(x)
    sin(y*x+1)
end

@code_warntype f(3.2)

Variables:
  #self#::#f
  x::Float64
  y::Union{Float64, Int64}
  fy::Float64
  #temp#@_5::Union{Float64, Int64}
  #temp#@_6::Core.MethodInstance
  #temp#@_7::Float64
  #temp#@_8::Float64

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location In[13] pos 1
      # meta: location float.jl < 491
      fy::Float64 = (Base.sitofp)(Float64, 0)::Float64
      # meta: pop location
      unless (Base.or_int)((Base.lt_float)(x::Float64, fy::Float64)::Bool, (Base.and_int)((Base.and_int)((Base.eq_float)(x::Float64, fy::Float64)::Bool, (Base.lt_float)(fy::Float64, 9.223372036854776e18)::Bool)::Bool, (Base.slt_int)((Base.fptosi)(Int64, fy::Float64)::Int64, 0)::Bool)::Bool)::Bool goto 9
      #temp#@_5::Union{Float64, Int64} = 0
      goto 11
      9: 
      #temp#@_5::Union{Float64, Int64} = x::Float64
      11: 
      # meta: pop location
      $(Expr(:inbounds, :pop))
      y::Union{Float64, Int64} = #temp#@_5::Union{Float64, Int64} # line 5:
      unless (y::Union{Float64, Int64} 

In [14]:
funcA() = sum([i^2 for i in 1:10000 if i%2==0]);
funcB() = sum(i^2 for i in 1:10000 if i%2==0);

In [15]:
println(@time sum([i^2 for i in 1:10000 if i%2==0]);)
println(@time sum(i^2 for i in 1:10000 if i%2==0);)
println(@time funcA();)
println(@time funcB();)

  0.083004 seconds (20.35 k allocations: 1.653 MiB)
166716670000
  0.078817 seconds (23.68 k allocations: 1.307 MiB)
166716670000
  0.069434 seconds (24.55 k allocations: 1.507 MiB)
166716670000
  0.065371 seconds (24.26 k allocations: 1.341 MiB)
166716670000


In [17]:
@time begin
    s=0
    for i in 1:10000
        if i%2==0
            s+=i^2
        end
    end
end

  0.000161 seconds (9.98 k allocations: 155.984 KiB)
